In [40]:
import numpy as np
import pandas as pd
import os
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
from sklearn.preprocessing import minmax_scale
import IPython.display as ipd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from scipy.integrate import cumulative_trapezoid
from scipy.signal import lfilter
import random


# Read data CSV (as per the structure you provided)
df = pd.read_csv('/kaggle/input/dysarthria-detection/torgo_data/data.csv')
df['filename'] = df['filename'].apply(lambda x: os.path.join('/kaggle/input/dysarthria-detection',x))

def compute_lpc_residual(signal, fs, order=12):
    preemphasized_signal = librosa.effects.preemphasis(signal)
    lpc_coeffs = librosa.lpc(preemphasized_signal, order=order)
    lpc_residual = lfilter([1] + -lpc_coeffs[1:], 1, preemphasized_signal)
    return lpc_residual



from scipy.signal.windows import hann

def extract_gvv_features(signal, fs, order_lpc=24, plot=False):
    signal = signal - np.mean(signal)  # Centering
    pre_emphasis_coeff = 1.0
    pre_sig = lfilter([1, -pre_emphasis_coeff], [1], signal)

    # Apply Hanning window
    win = hann(len(pre_sig))
    sig_lpc = pre_sig * win

    try:
        # LPC coefficients
        a = librosa.lpc(sig_lpc, order=order_lpc)
        
        # Inverse filtering to get excitation
        ex = lfilter([0] + [-1 * coef for coef in a[1:]], [1], sig_lpc)
        
        # LP residual: original - excitation
        lp_residual = sig_lpc - ex
        
        # GVV computation by integration
        gvv = cumulative_trapezoid(lp_residual, dx=1/fs, initial=0)


        if plot:
            frame_duration_ms = 30  # 30 milliseconds
            frame_length = int(fs * frame_duration_ms / 1000)
            start = random.randint(0, len(signal) - frame_length)
            frame = signal[start:start + frame_length]
            ppp = gvv[start:start + frame_length]
            ppp2 = lp_residual[start:start + frame_length]
    
            plt.figure(figsize=(10, 6))
            plt.plot(ppp, label='GVV')
            plt.title('Glottal Volume Velocity (GVV)')
            plt.xlabel('Frame Index')
            plt.ylabel('GVV')
            plt.show()

            plt.figure(figsize=(10, 6))
            plt.plot(ppp2)
            plt.title('LP residual')
            plt.xlabel('Frame Index')
            plt.ylabel('LP')
            plt.legend()
            plt.show()


        # GVV-based features
        opening_phase_duration = np.sum(gvv > 0) * (1/fs)
        closing_phase_duration = np.sum(gvv < 0) * (1/fs)

        opening_slope = np.mean(np.diff(gvv[gvv > 0])) if np.any(gvv > 0) else 0
        closing_slope = np.mean(np.diff(gvv[gvv < 0])) if np.any(gvv < 0) else 0
        slope_ratio = opening_slope / (closing_slope if closing_slope != 0 else 1)

        return [opening_phase_duration, closing_phase_duration, opening_slope, closing_slope, slope_ratio]
    
    except Exception as e:
        print(f"GVV feature extraction failed: {e}")
        return [0, 0, 0, 0, 0]

    
def feature_extraction_only_gvv(df):
    features = []
    for i, record in tqdm(df.iterrows(), total=df.shape[0]):
        try:
            speech, fs = librosa.load(record['filename'])
            # if record['is_dysarthria'] != "non_dysarthria":
            #     print(record['is_dysarthria'])
            #     gvv_features = extract_gvv_features(speech, fs, plot=True)
            # else:
            gvv_features = extract_gvv_features(speech, fs, plot=False)
            all_features = np.concatenate([gvv_features])
            features.append(np.append(all_features, [record['is_dysarthria'], record['gender']]))  
        except Exception as e:
            print(f"Error processing {record['filename']}: {e}")

    column_names = [
        'Opening_Phase_Duration', 'Closing_Phase_Duration', 'Opening_Phase_Slope', 'Closing_Phase_Slope', 'Slope_Ratio', 'class', 'gender'
    ]
    return pd.DataFrame(features, columns=column_names)

data_with_feat_gvv = feature_extraction_only_gvv(df)
data_with_feat_gvv['class'] = data_with_feat_gvv['class'].replace('non_dysarthria', 0)
data_with_feat_gvv['class'] = data_with_feat_gvv['class'].replace('dysarthria', 1)
data_with_feat_gvv['gender'] = data_with_feat_gvv['gender'].replace('male', 1)
data_with_feat_gvv['gender'] = data_with_feat_gvv['gender'].replace('female', 0)
data_with_feat_gvv

X_gvv = data_with_feat_gvv.drop(columns=['class'])
X_gvv.columns = X_gvv.columns.astype(str)
y_gvv = data_with_feat_gvv['class']
X_gvv = X_gvv.astype(float)

X_train_gvv, X_test_gvv, y_train_gvv, y_test_gvv = train_test_split(X_gvv, y_gvv, test_size=0.2, stratify=y_gvv, random_state=42)

X_train_gvv.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train_gvv.fillna(X_train_gvv.mean(), inplace=True)
# X_train_gvv = X_train_gvv.loc[:, X_train_gvv.nunique() > 1] 

X_train_gvv = X_train_gvv.astype(float)
y_train_gvv = y_train_gvv.astype(int)

X_test_gvv.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test_gvv.fillna(X_test_gvv.mean(), inplace=True) 


scaler = StandardScaler()
X_train_gvv = scaler.fit_transform(X_train_gvv)
X_test_gvv = scaler.transform(X_test_gvv)

from sklearn.model_selection import GridSearchCV
param_grid = [
    {'C': [0.5, 1, 10, 100, 1000],
     'gamma': [100, 10, 1, 0.1, 0.001, 0.00001, 0.000001],
     'kernel': ['rbf'],
    }
]

optional_params = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', verbose=0)
optional_params.fit(X_train_gvv, y_train_gvv)
print("Best parameters for original dataset:")
print(optional_params.best_params_)

 30%|██▉       | 595/2000 [00:11<00:31, 44.48it/s]/tmp/ipykernel_31/4134614764.py:101: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, fs = librosa.load(record['filename'])
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
 30%|███       | 609/2000 [00:11<00:27, 50.12it/s]

Error processing /kaggle/input/dysarthria-detection/torgo_data/dysarthria_female/F01_Session1_0068.wav: 


100%|██████████| 2000/2000 [00:43<00:00, 46.23it/s]
/tmp/ipykernel_31/4134614764.py:119: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_with_feat_gvv['class'] = data_with_feat_gvv['class'].replace('dysarthria', 1)
/tmp/ipykernel_31/4134614764.py:121: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_with_feat_gvv['gender'] = data_with_feat_gvv['gender'].replace('female', 0)


Best parameters for original dataset:
{'C': 1, 'gamma': 10, 'kernel': 'rbf'}


In [41]:
model_gvv = SVC(kernel='rbf', gamma=10, C=1)
model_gvv.fit(X_train_gvv, y_train_gvv)

from sklearn.metrics import accuracy_score

predictions = model_gvv.predict(X_test_gvv) 
print(100 * accuracy_score(y_test_gvv, predictions), "% accuracy")

76.75 % accuracy


In [46]:
import numpy as np
import pandas as pd
import os
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
from sklearn.preprocessing import minmax_scale
import IPython.display as ipd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from scipy.integrate import cumulative_trapezoid
from scipy.signal import lfilter
import random


# Read data CSV (as per the structure you provided)
df = pd.read_csv('/kaggle/input/dysarthria-detection/torgo_data/data.csv')
df['filename'] = df['filename'].apply(lambda x: os.path.join('/kaggle/input/dysarthria-detection',x))

def compute_lpc_residual(signal, fs, order=12):
    preemphasized_signal = librosa.effects.preemphasis(signal)
    lpc_coeffs = librosa.lpc(preemphasized_signal, order=order)
    lpc_residual = lfilter([1] + -lpc_coeffs[1:], 1, preemphasized_signal)
    return lpc_residual



from scipy.signal.windows import hann

from scipy.stats import skew, kurtosis
from scipy.signal import periodogram
from sklearn.preprocessing import normalize

def extract_gvv_features(signal, fs, order_lpc=24, plot=False):
    signal = signal - np.mean(signal)  # Centering
    pre_emphasis_coeff = 1.0
    pre_sig = lfilter([1, -pre_emphasis_coeff], [1], signal)

    win = hann(len(pre_sig))
    sig_lpc = pre_sig * win

    try:
        a = librosa.lpc(sig_lpc, order=order_lpc)
        ex = lfilter([0] + [-1 * coef for coef in a[1:]], [1], sig_lpc)
        lp_residual = sig_lpc - ex
        gvv = cumulative_trapezoid(lp_residual, dx=1/fs, initial=0)

        # Optional visualization
        if plot:
            frame_duration_ms = 30
            frame_length = int(fs * frame_duration_ms / 1000)
            start = random.randint(0, len(signal) - frame_length)
            frame = signal[start:start + frame_length]
            ppp = gvv[start:start + frame_length]
            ppp2 = lp_residual[start:start + frame_length]

            plt.figure(figsize=(10, 6))
            plt.plot(ppp, label='GVV')
            plt.title('Glottal Volume Velocity (GVV)')
            plt.xlabel('Frame Index')
            plt.ylabel('GVV')
            plt.show()

            plt.figure(figsize=(10, 6))
            plt.plot(ppp2)
            plt.title('LP residual')
            plt.xlabel('Frame Index')
            plt.ylabel('LP')
            plt.legend()
            plt.show()

        # Higher-order statistics
        gvv_skewness = skew(gvv)
        gvv_kurtosis = kurtosis(gvv)
        rms_energy = np.sqrt(np.mean(gvv**2))

        # Energy entropy
        gvv_energy = gvv**2
        gvv_energy = gvv_energy / np.sum(gvv_energy)  # Normalize
        energy_entropy = -np.sum(gvv_energy * np.log2(gvv_energy + 1e-10))  # Avoid log(0)

        return [gvv_skewness, gvv_kurtosis, energy_entropy, rms_energy]
    
    except Exception as e:
        print(f"GVV feature extraction failed: {e}")
        return [0, 0, 0, 0]


    
def feature_extraction_only_gvv(df):
    features = []
    for i, record in tqdm(df.iterrows(), total=df.shape[0]):
        try:
            speech, fs = librosa.load(record['filename'])
            # if record['is_dysarthria'] != "non_dysarthria":
            #     print(record['is_dysarthria'])
            #     gvv_features = extract_gvv_features(speech, fs, plot=True)
            # else:
            gvv_features = extract_gvv_features(speech, fs, plot=False)
            all_features = np.concatenate([gvv_features])
            features.append(np.append(all_features, [record['is_dysarthria'], record['gender']]))  
        except Exception as e:
            print(f"Error processing {record['filename']}: {e}")

    column_names = [
        'GVV_Skewness', 'GVV_Kurtosis', 'GVV_EnergyEntropy', 'GVV_RMSEnergy', 'class', 'gender'
    ]

    return pd.DataFrame(features, columns=column_names)

data_with_feat_gvv = feature_extraction_only_gvv(df)
data_with_feat_gvv['class'] = data_with_feat_gvv['class'].replace('non_dysarthria', 0)
data_with_feat_gvv['class'] = data_with_feat_gvv['class'].replace('dysarthria', 1)
data_with_feat_gvv['gender'] = data_with_feat_gvv['gender'].replace('male', 1)
data_with_feat_gvv['gender'] = data_with_feat_gvv['gender'].replace('female', 0)
data_with_feat_gvv

X_gvv = data_with_feat_gvv.drop(columns=['class'])
X_gvv.columns = X_gvv.columns.astype(str)
y_gvv = data_with_feat_gvv['class']
X_gvv = X_gvv.astype(float)

X_train_gvv, X_test_gvv, y_train_gvv, y_test_gvv = train_test_split(X_gvv, y_gvv, test_size=0.2, stratify=y_gvv, random_state=42)

X_train_gvv.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train_gvv.fillna(X_train_gvv.mean(), inplace=True)
# X_train_gvv = X_train_gvv.loc[:, X_train_gvv.nunique() > 1] 

X_train_gvv = X_train_gvv.astype(float)
y_train_gvv = y_train_gvv.astype(int)

X_test_gvv.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test_gvv.fillna(X_test_gvv.mean(), inplace=True) 


scaler = StandardScaler()
X_train_gvv = scaler.fit_transform(X_train_gvv)
X_test_gvv = scaler.transform(X_test_gvv)

from sklearn.model_selection import GridSearchCV
param_grid = [
    {'C': [0.5, 1, 10, 100, 1000],
     'gamma': [100, 10, 1, 0.1, 0.001, 0.00001, 0.000001],
     'kernel': ['rbf'],
    }
]

optional_params = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy', verbose=0)
optional_params.fit(X_train_gvv, y_train_gvv)
print("Best parameters for original dataset:")
print(optional_params.best_params_)

 30%|██▉       | 599/2000 [00:13<00:32, 42.55it/s]/tmp/ipykernel_31/320939344.py:99: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, fs = librosa.load(record['filename'])
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
 30%|███       | 606/2000 [00:13<00:31, 44.25it/s]

Error processing /kaggle/input/dysarthria-detection/torgo_data/dysarthria_female/F01_Session1_0068.wav: 


100%|██████████| 2000/2000 [00:48<00:00, 41.64it/s]
/tmp/ipykernel_31/320939344.py:118: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_with_feat_gvv['class'] = data_with_feat_gvv['class'].replace('dysarthria', 1)
/tmp/ipykernel_31/320939344.py:120: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_with_feat_gvv['gender'] = data_with_feat_gvv['gender'].replace('female', 0)


Best parameters for original dataset:
{'C': 1, 'gamma': 1, 'kernel': 'rbf'}


In [47]:
model_gvv = SVC(kernel='rbf', gamma=1, C=1)
model_gvv.fit(X_train_gvv, y_train_gvv)

from sklearn.metrics import accuracy_score

predictions = model_gvv.predict(X_test_gvv) 
print(100 * accuracy_score(y_test_gvv, predictions), "% accuracy")

83.0 % accuracy
